## Temps passé par utilisateur sur les réseaux sociaux

In [ ]:
#Importation de librairies
import pandas as pd

In [ ]:
repertoire = "./Data/Reseaux/"
fichier_utilisateurs = repertoire + "utilisateurs.csv"
fichier_temps_passe = repertoire + "reseaux_sociaux.csv"

In [ ]:
#1. Récupérer les données à partir des DataFrames
df_utilisateurs = pd.read_csv(fichier_utilisateurs, encoding="latin-1")

df_donnees = pd.read_csv(fichier_temps_passe, encoding="latin-1")

display(df_utilisateurs)
display(df_donnees)

In [ ]:
# 2. & 3. Renommer les colonnes
df_utilisateurs.rename(columns={'id': 'id_utilisateur'}, inplace=True)

df_donnees.rename(columns={'id': 'id_data'}, inplace=True)

display(df_utilisateurs)
display(df_donnees)

In [ ]:
display(df_utilisateurs.head())

display(df_donnees.head())

In [ ]:
# 4. Mergez les 2 dataframes
df_merge = df_utilisateurs.merge(df_donnees, on="adresse IP", how="left")

df_merge

In [ ]:
# 5. Calculez le nombre d'heures total passé par utilisateur
df_merge.groupby(["id_utilisateur", "nom", "prénom"]).agg(temps_total=("temps passé", "sum"))

In [ ]:
# 6. Calculez le nombre d'heures total passé par utilisateur
df_tps_passe2 = df_merge.groupby(["nom", "prénom"]) \
    .agg(temps_total=("temps passé", "sum"))
df_tps_passe2

In [ ]:
# 7. Calculez le nombre d'heures total passé sur chaque réseau social
df_classement_site = df_merge.groupby(["site"]) \
    .agg(temps_total=("temps passé", "sum")) \
    .sort_values("temps_total", ascending=False)

df_classement_site

In [ ]:
df_classement_site.index.to_list()

In [ ]:
df_classement_site["temps_total"].to_list()

### Génération de courbes et histogrammes

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

x = df_classement_site.index
y = df_classement_site["temps_total"]

#met les labels verticalement
plt.xticks(ticks=range(len(x)), labels=x, rotation=90)
plt.plot(x, y, label='ma courbe')

plt.title('ma courbe')
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

sites = df_classement_site.index
tps = df_classement_site["temps_total"]

fig, ax = plt.subplots()
ax.bar(sites, tps)

#met les labels verticalement
plt.xticks(ticks=range(len(tps)), labels=sites, rotation=90)
plt.show()

In [ ]:
df_classement_site

In [ ]:
# 8. Au dataframe obtenu précédement, ajoutez le pourcentage de temps passé pour chaque réseau social
temps_total_tous_sites = df_classement_site["temps_total"].sum()

def calculer_pourcentage_temps(temps):
    return round(temps*100 / temps_total_tous_sites, 2)


df_classement_site["pourcentage temps"] = df_classement_site["temps_total"].apply(calculer_pourcentage_temps)
#df_classement_site["pourcentage temps"] = round((df_classement_site["temps_total"] / temps_total_tous_sites) * 100, 2)

df_classement_site

In [ ]:
df_merge.info()

In [ ]:
df_merge_cp = df_merge.copy()

In [ ]:
df_merge_cp["année de naissance"] = df_merge_cp["date de naissance"].apply(lambda date: pd.to_datetime(date, format="%Y-%m-%d").year)
df_merge_cp

In [ ]:
# 9. Au dataframe `df_merge`, ajoutez une colonne `année de naissance`

df_merge["année de naissance"] = df_merge["date de naissance"].apply(lambda date: int(date[:4]))
df_merge

#def transformer_date_naissance_annee(date_de_naissance):
#    return int(date_de_naissance[:4])

#df_merge["année de naissance"] = df_merge["date de naissance"].apply(transformer_date_naissance_annee)
#df_merge

In [ ]:
# 10. créez un tableau croisé dynamique
df_classement_site_annee_naissance = df_merge \
    .pivot_table(index=["site"], columns=["année de naissance"], values="temps passé", aggfunc="mean")
df_classement_site_annee_naissance

In [ ]:
from statistics import mean

df_classement_site_annee_naissance2 = df_merge \
    .pivot_table(index=["site"], columns=["année de naissance"], values="temps passé", aggfunc=lambda x:round(mean(x), 2))
df_classement_site_annee_naissance2

In [ ]:
df_merge \
    .pivot_table(index=["site", "année de naissance"], values="temps passé", aggfunc=lambda x:round(mean(x), 2)) \
    .reset_index()